In [1]:
#饭店流量数据
import pandas as pd
air_visit = pd.read_csv('air_visit_data.csv')
air_visit.index = pd.to_datetime(air_visit['visit_date'])
air_visit.head()


,air_store_id,visit_date,visitors
visit_date,,,
2016-01-13,air_ba937bf13d40fb24,2016-01-13,25
2016-01-14,air_ba937bf13d40fb24,2016-01-14,32
2016-01-15,air_ba937bf13d40fb24,2016-01-15,29
2016-01-16,air_ba937bf13d40fb24,2016-01-16,22
2016-01-18,air_ba937bf13d40fb24,2016-01-18,6


In [2]:
# 按天来算
air_visit = air_visit.groupby('air_store_id').apply(lambda g: g['visitors'].resample('1d').sum()).reset_index()


In [3]:
# 缺失值填0n
air_visit['visit_date'] = air_visit['visit_date'].dt.strftime('%Y-%m-%d')
air_visit['was_nil'] = air_visit['visitors'].isnull()
air_visit['visitors'].fillna(0, inplace = True)
air_visit.head()

,air_store_id,visit_date,visitors,was_nil
0,air_00a91d42b08b08d9,2016-07-01,35,False
1,air_00a91d42b08b08d9,2016-07-02,9,False
2,air_00a91d42b08b08d9,2016-07-03,0,False
3,air_00a91d42b08b08d9,2016-07-04,20,False
4,air_00a91d42b08b08d9,2016-07-05,25,False


In [4]:
#日历
date_info = pd.read_csv('date_info.csv')
date_info.rename(columns={'holiday_flg':'is_holiday', 'calendar_date':'visit_date'}, inplace=True)
date_info['prev_day_is_holiday'] = date_info['is_holiday'].shift().fillna(0)
date_info['next_day_is_holiday'] = date_info['is_holiday'].shift(-1).fillna(0)
date_info.head()

,visit_date,day_of_week,is_holiday,prev_day_is_holiday,next_day_is_holiday
0,2016-01-01,Friday,1,0.0,1.0
1,2016-01-02,Saturday,1,1.0,1.0
2,2016-01-03,Sunday,1,1.0,0.0
3,2016-01-04,Monday,0,1.0,0.0
4,2016-01-05,Tuesday,0,0.0,0.0


In [5]:
#地区
air_store_info = pd.read_csv('air_store_info.csv')
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [6]:
#测试集
import numpy as np
submission = pd.read_csv('sample_submission.csv')

submission['air_store_id'] = submission['id'].str.slice(0, 20)
submission['visit_date'] = submission['id'].str.slice(21)
submission['is_test'] = True
submission['visitors'] = np.nan
submission['test_number'] = range(len(submission))
submission.head()

,id,visitors,air_store_id,visit_date,is_test,test_number
0,air_00a91d42b08b08d9_2017-04-23,NaN,air_00a91d42b08b08d9,2017-04-23,True,0
1,air_00a91d42b08b08d9_2017-04-24,NaN,air_00a91d42b08b08d9,2017-04-24,True,1
2,air_00a91d42b08b08d9_2017-04-25,NaN,air_00a91d42b08b08d9,2017-04-25,True,2
3,air_00a91d42b08b08d9_2017-04-26,NaN,air_00a91d42b08b08d9,2017-04-26,True,3
4,air_00a91d42b08b08d9_2017-04-27,NaN,air_00a91d42b08b08d9,2017-04-27,True,4


In [7]:
#数据概况
print(air_visit.shape, submission.shape)
data = pd.concat((air_visit, submission.drop('id', axis = 'columns')))
print(data.shape)
data.head()

(296279, 4) (32019, 6)
(328298, 6)


,air_store_id,visit_date,visitors,was_nil,is_test,test_number
0,air_00a91d42b08b08d9,2016-07-01,35.0,False,NaN,NaN
1,air_00a91d42b08b08d9,2016-07-02,9.0,False,NaN,NaN
2,air_00a91d42b08b08d9,2016-07-03,0.0,False,NaN,NaN
3,air_00a91d42b08b08d9,2016-07-04,20.0,False,NaN,NaN
4,air_00a91d42b08b08d9,2016-07-05,25.0,False,NaN,NaN


In [8]:
data['is_test'].fillna(False, inplace = True)
print(date_info.shape, data.shape)
data = pd.merge(left=data, right= date_info, on='visit_date', how='left')

print(air_store_info.shape, data.shape)
data = pd.merge(left=data, right=air_store_info, on='air_store_id', how='left')
print(data.shape)

data['visitors'] = data['visitors'].astype(float)
data.head()

(517, 5) (328298, 6)
(829, 5) (328298, 10)
(328298, 14)


,air_store_id,visit_date,visitors,was_nil,is_test,test_number,day_of_week,is_holiday,prev_day_is_holiday,next_day_is_holiday,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
1,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
2,air_00a91d42b08b08d9,2016-07-03,0.0,False,False,NaN,Sunday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
3,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
4,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595


In [9]:
#天气
import glob

weather_dfs = []
for path in glob.glob('/hotel/Weather/*.csv'):
	weather_df = pd.read_csv(path)
	weather_df['station_id'] = path.split('\\')[-1].rstrip('.csv')
	weather_dfs.append(weather_df)

weather = pd.concat(weather_dfs, axis=0)
weather.rename(columns={'calendar_date':'visit_date'}, inplace=True)
weather.head()

,visit_date,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,solar_radiation,deepest_snowfall,total_snowfall,avg_wind_speed,avg_vapor_pressure,avg_local_pressure,avg_humidity,avg_sea_pressure,cloud_cover,station_id
0,2016-01-01,6.0,11.0,0.7,0.0,9.1,NaN,NaN,NaN,2.9,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
1,2016-01-02,4.7,10.5,0.0,0.0,6.8,NaN,NaN,NaN,1.4,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
2,2016-01-03,7.0,13.8,1.9,0.0,8.6,NaN,NaN,NaN,1.4,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
3,2016-01-04,8.8,14.7,2.7,0.0,5.3,NaN,NaN,NaN,2.2,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
4,2016-01-05,8.9,14.1,3.4,0.0,4.5,NaN,NaN,NaN,2.7,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai


In [10]:
# 用各个小地方数据求出平均气温

means = weather.groupby('visit_date')[['avg_temperature', 'precipitation']].mean().reset_index()
means.rename(columns={'avg_temperature': 'global_avg_temperature', 'precipitation': 'global_precipitation'}, inplace=True)
weather = pd.merge(left=weather, right=means, on='visit_date', how='left')
weather['avg_temperature'].fillna(weather['global_avg_temperature'], inplace=True)
weather['precipitation'].fillna(weather['global_precipitation'], inplace=True)

weather_df = weather[['visit_date','avg_temperature','precipitation']]
print(weather_df.shape)

weather = weather_df.groupby('visit_date').mean()
weather.head()

(859771, 3)


,avg_temperature,precipitation
visit_date,,
2016-01-01,2.868353,0.564662
2016-01-02,5.279225,2.341998
2016-01-03,6.589978,1.750616
2016-01-04,5.857883,1.644946
2016-01-05,4.556850,3.193625


In [11]:
print(data.shape, weather.shape)
data = pd.merge(left=data, right=weather, on='visit_date', how='left')
print(data.shape)
data.head()

(328298, 14) (517, 2)
(328298, 16)


,air_store_id,visit_date,visitors,was_nil,is_test,test_number,day_of_week,is_holiday,prev_day_is_holiday,next_day_is_holiday,air_genre_name,air_area_name,latitude,longitude,avg_temperature,precipitation
0,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,23.633584,0.704620
1,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,24.453341,8.149236
2,air_00a91d42b08b08d9,2016-07-03,0.0,False,False,NaN,Sunday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,24.087931,7.297010
3,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,22.918966,4.565769
4,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,22.070874,1.375406


In [12]:
#信息数据
data['visit_date'] = pd.to_datetime(data['visit_date'])
data['visits_date'] = data['visit_date']
data = data.set_index('visits_date')
data.sort_values(['air_store_id', 'visit_date'], inplace=True)
data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,test_number,day_of_week,is_holiday,prev_day_is_holiday,next_day_is_holiday,air_genre_name,air_area_name,latitude,longitude,avg_temperature,precipitation
visits_date,,,,,,,,,,,,,,,,
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,23.633584,0.704620
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,24.453341,8.149236
2016-07-03,air_00a91d42b08b08d9,2016-07-03,0.0,False,False,NaN,Sunday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,24.087931,7.297010
2016-07-04,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,22.918966,4.565769
2016-07-05,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,22.070874,1.375406


In [13]:
#离群点处理 正太分布，选择了1.96这个值。
def find_outliers(series):
	return (series - series.mean()) > 1.96 * series.std()
def cap_values(series):
	outliers = find_outliers(series)
	max_val = series[~outliers].max()
	series[outliers] = max_val
	return series

stores = data.groupby('air_store_id')
data['is_outlier'] = stores.apply(lambda b:find_outliers(b['visitors'])).values
data['visitors_capped'] = stores.apply(lambda b:cap_values(b['visitors'])).values
data['visitors_capped_log1p'] = np.log1p(data['visitors_capped']) # log1p: = log(x+1)

In [14]:
data['is_weekend'] = data['day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
data['day_of_month'] = data['visit_date'].dt.day
data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,test_number,day_of_week,is_holiday,prev_day_is_holiday,next_day_is_holiday,...,air_area_name,latitude,longitude,avg_temperature,precipitation,is_outlier,visitors_capped,visitors_capped_log1p,is_weekend,day_of_month
visits_date,,,,,,,,,,,,,,,,,,,,,
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,23.633584,0.704620,False,35.0,3.583519,0,1
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,24.453341,8.149236,False,9.0,2.302585,1,2
2016-07-03,air_00a91d42b08b08d9,2016-07-03,0.0,False,False,NaN,Sunday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,24.087931,7.297010,False,0.0,0.000000,1,3
2016-07-04,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,22.918966,4.565769,False,20.0,3.044522,0,4
2016-07-05,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,22.070874,1.375406,False,25.0,3.258097,0,5


In [ ]:

from scipy import optimize
 
 
def calc_shifted_ewm(series, alpha, adjust=True):
    return series.shift().ewm(alpha=alpha, adjust=adjust).mean()
 
 
def find_best_signal(series, adjust=False, eps=10e-5):
    
    def f(alpha):
        shifted_ewm = calc_shifted_ewm(series=series, alpha=min(max(alpha, 0), 1), adjust=adjust)
        corr = np.mean(np.power(series - shifted_ewm, 2))
        return corr
     
    res = optimize.differential_evolution(func=f, bounds=[(0 + eps, 1 - eps)])
    
    return calc_shifted_ewm(series=series, alpha=res['x'][0], adjust=adjust)
 
 
roll = data.groupby(['air_store_id', 'day_of_week']).apply(lambda g: find_best_signal(g['visitors_capped']))
data['optimized_ewm_by_air_store_id_&_day_of_week'] = roll.sort_index(level=['air_store_id', 'visit_date']).values
 
roll = data.groupby(['air_store_id', 'is_weekend']).apply(lambda g: find_best_signal(g['visitors_capped']))
data['optimized_ewm_by_air_store_id_&_is_weekend'] = roll.sort_index(level=['air_store_id', 'visit_date']).values
 
roll = data.groupby(['air_store_id', 'day_of_week']).apply(lambda g: find_best_signal(g['visitors_capped_log1p']))
data['optimized_ewm_log1p_by_air_store_id_&_day_of_week'] = roll.sort_index(level=['air_store_id', 'visit_date']).values
 
roll = data.groupby(['air_store_id', 'is_weekend']).apply(lambda g: find_best_signal(g['visitors_capped_log1p']))
data['optimized_ewm_log1p_by_air_store_id_&_is_weekend'] = roll.sort_index(level=['air_store_id', 'visit_date']).values
 


In [ ]:
def extract_precedent_statistics(df, on, group_by):
    
    df.sort_values(group_by + ['visit_date'], inplace=True)
    
    groups = df.groupby(group_by, sort=False)
    
    stats = {
        'mean': [],
        'median': [],
        'std': [],
        'count': [],
        'max': [],
        'min': []
    }
    
    exp_alphas = [0.1, 0.25, 0.3, 0.5, 0.75]
    stats.update({'exp_{}_mean'.format(alpha): [] for alpha in exp_alphas})
    
    for _, group in groups:
        
        shift = group[on].shift()
        roll = shift.rolling(window=len(group), min_periods=1)
        
        stats['mean'].extend(roll.mean())
        stats['median'].extend(roll.median())
        stats['std'].extend(roll.std())
        stats['count'].extend(roll.count())
        stats['max'].extend(roll.max())
        stats['min'].extend(roll.min())
        
        for alpha in exp_alphas:
            exp = shift.ewm(alpha=alpha, adjust=False)
            stats['exp_{}_mean'.format(alpha)].extend(exp.mean())
    
    suffix = '_&_'.join(group_by)
    
    for stat_name, values in stats.items():
        df['{}_{}_by_{}'.format(on, stat_name, suffix)] = values
extract_precedent_statistics(
    df=data,
    on='visitors_capped',
    group_by=['air_store_id', 'day_of_week']
)
extract_precedent_statistics(
    df=data,
    on='visitors_capped',
    group_by=['air_store_id', 'is_weekend']
)
extract_precedent_statistics(
    df=data,
    on='visitors_capped',
    group_by=['air_store_id']
)
extract_precedent_statistics(
    df=data,
    on='visitors_capped_log1p',
    group_by=['air_store_id', 'day_of_week']
)
extract_precedent_statistics(
    df=data,
    on='visitors_capped_log1p',
    group_by=['air_store_id', 'is_weekend']
)
extract_precedent_statistics(
    df=data,
    on='visitors_capped_log1p',
    group_by=['air_store_id']
)
data.sort_values(['air_store_id', 'visit_date']).head()